In [38]:
# imports
import os
import requests
import pandas as pd
import time

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
api_key = "Bearer API KEY"
headers = {
        "Accept": "application/json",
        "Authorization": api_key,
        "X-Places-Api-Version": "2025-06-17"
    }
fsq_url = "https://places-api.foursquare.com/places/search"
results = []
for station in stations:
        lat = station['latitude']
        lon = station['longitude']
for index, row in df.iterrows():
      params = {
        "ll": f"{row['latitude']},{row['longitude']}",
        "radius": 1000,
        "limit": 50
    }
response = requests.get(fsq_url, headers=headers, params=params)
fsq_response = response.json()
station_result = {
        "station_name": station['name'],
        "latitude": lat,
        "longitude": lon,
        "nearby_places": []
    }
station_result

{'station_name': 'Alexandre Parodi - Quai de Valmy',
 'latitude': 48.879359419425,
 'longitude': 2.3665961623192,
 'nearby_places': []}

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [51]:
poi_list = []

for place in fsq_response['results']:
    name = place.get('name')
    latitude = place.get('latitude')
    longitude = place.get('longitude')
    distance = place.get('distance')

    # Get first category details
    categories = place.get('categories', [])
    category_name = categories[0]['name'].lower() if categories else None
    fsq_category_id = categories[0]['fsq_category_id'] if categories else None

    # Normalize to group
    if "restaurant" in category_name:
        category_group = "restaurant"
    elif "bar" in category_name:
        category_group = "bar"
    elif "cafe" in category_name or "café" in category_name:
        category_group = "cafe"
    elif 'bakery' in category_name:
        category_group = 'bakery'
    elif 'hotel' in category_name:
        category_group = 'hotel'
    elif 'museum' in category_name:
        category_group = 'museum'
    elif 'park' in category_name:
        category_group = 'park'
    elif 'supermarket' in category_name or 'grocery' in category_name:
        category_group = 'store'
    elif 'pharmacy' in category_name:
        category_group = 'pharmacy'
    elif 'church' in category_name or 'mosque' in category_name or 'temple' in category_name:
        category_group = 'religious'
    elif 'gym' in category_name or 'fitness' in category_name:
        category_group = 'fitness'
    else:
        category_group = 'other'

    # Location info
    location = place.get('location', {})
    address = location.get('formatted_address')
    region = location.get('region')

    poi_list.append({
        'name': name,
        'category': category_name,
        'category_group': category_group,
        'latitude': latitude,
        'longitude': longitude,
        'distance': distance,
        'address': address,
        'region': region,
        'fsq_category_id': fsq_category_id
    })

In [23]:
station_result = {
        "station_name": station['name'],
        "latitude": lat,
        "longitude": lon,
        "nearby_places": []
    }
station_result

{'station_name': 'Alexandre Parodi - Quai de Valmy',
 'latitude': 48.879359419425,
 'longitude': 2.3665961623192,
 'nearby_places': []}

In [52]:
poi_list = []

for place in fsq_response['results']:
    name = place.get('name')
    latitude = place.get('latitude')
    longitude = place.get('longitude')
    distance = place.get('distance')

    # Get first category details
    categories = place.get('categories', [])
    category_name = categories[0]['name'].lower() if categories else None
    fsq_category_id = categories[0]['fsq_category_id'] if categories else None

    # Normalize to group
    if "restaurant" in category_name:
        category_group = "restaurant"
    elif "bar" in category_name:
        category_group = "bar"
    elif "cafe" in category_name or "café" in category_name:
        category_group = "cafe"
    else:
        category_group = "other"

    # Location info
    location = place.get('location', {})
    address = location.get('formatted_address')
    region = location.get('region')

    poi_list.append({
        'name': name,
        'category': category_name,
        'category_group': category_group,
        'latitude': latitude,
        'longitude': longitude,
        'distance': distance,
        'address': address,
        'region': region,
        'fsq_category_id': fsq_category_id
    })

Put your parsed results into a DataFrame

In [54]:
# Convert to DataFrame
df_pois = pd.DataFrame(poi_list)

# Categorize into general POI types
def categorize(poi):
    if poi is None:
        return None
    category = poi.lower()
    if 'restaurant' in category:
        return 'restaurant'
    elif 'bar' in category:
        return 'bar'
    elif 'cafe' in category or 'coffee' in category:
        return 'cafe'
    elif 'bakery' in category:
        return 'bakery'
    elif 'hotel' in category:
        return 'hotel'
    elif 'museum' in category:
        return 'museum'
    elif 'park' in category:
        return 'park'
    elif 'supermarket' in category or 'grocery' in category:
        return 'store'
    elif 'pharmacy' in category:
        return 'pharmacy'
    elif 'church' in category or 'mosque' in category or 'temple' in category:
        return 'religious'
    elif 'gym' in category or 'fitness' in category:
        return 'fitness'
    else:
        return 'other'

df_pois['category_type'] = df_pois['category'].apply(categorize)
df_pois.head()

,name,category,category_group,latitude,longitude,distance,address,region,fsq_category_id,category_type
0,Papelli,pizzeria,other,48.871671,2.356960,266,"61 rue du Faubourg Saint-Martin, 75010 Paris",Île-de-France,4bf58dd8d48988d1ca941735,other
1,Pâtisserie Tholoniat,dessert shop,other,48.871834,2.356693,284,54-56 rue du Château d'Eau (Rue du faubourg Sa...,Île-de-France,4bf58dd8d48988d1d0941735,other
2,New Dehli,indian restaurant,restaurant,48.871066,2.356008,349,"4 Passage Brady, 75010 Paris",Île-de-France,4bf58dd8d48988d10f941735,restaurant
3,BMK Paris-Bamako,african restaurant,restaurant,48.874646,2.356549,416,"14 rue de la Fidélité, 75010 Paris",Île-de-France,4bf58dd8d48988d1c8941735,restaurant
4,Boulangerie Julhès,bakery,other,48.872290,2.354474,447,"56 rue du Faubourg Saint-Denis, 75010 Paris",Île-de-France,4bf58dd8d48988d16a941735,bakery


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Since I was unable to access Yelp, I conducted some filtering of the available data on the places of interest to establish the accessibility of bikes by these POI based on distance. The DataFrame showed that Papelli was closest by distance to a citybike place, while Médiathèque Françoise Sagan was the farthest. This was based on the selection limit of 50 rows.

Get the top 10 restaurants according to their rating

In [57]:
# Sort by distance in descending order
df_sorted = df_pois.sort_values(by='distance', ascending=True)

# Display sorted DataFrame
closest_poi = df_sorted[['name', 'distance']]
closest_poi

,name,distance
0,Papelli,266
1,Pâtisserie Tholoniat,284
2,New Dehli,349
3,BMK Paris-Bamako,416
11,Le Bec Fin,421
6,Distrito Francés Saint Martin,429
14,Sheezan,439
12,Théâtre Antoine,442
13,Grand Café d'Athènes,443
5,Au Faubourg,446


In [58]:
df_pois.to_csv('df_pois.csv', index=False)